In [1]:
import datasets
from transformers import DecisionTransformerConfig, DecisionTransformerModel, TrainingArguments, Trainer

from game import PlayerDecision, initialize_game_state

import numpy as np
import random
import torch

# Initializing a DecisionTransformer configuration

state_size = len(initialize_game_state().to_state_array(0))
action_size = len(PlayerDecision(0).to_state_array())

In [2]:
ds = datasets.load_from_disk("data/dataset3")
				
ds["actions"][0][0].__len__()

9

In [3]:
# trainer.train()
ds


Dataset({
    features: ['states', 'actions', 'rewards', 'dones'],
    num_rows: 1000
})

In [4]:
dataset = datasets.load_dataset("edbeeching/decision_transformer_gym_replay", "halfcheetah-expert-v2")
dataset['train'][0]["dones"].__len__()

1000

In [5]:
split_ds = ds.train_test_split(test_size=0.1)

In [10]:
TARGET_RETURN = 20

class MyDataCollator:
    # This class maps "states", "actions", "rewards", "done"
    # into 
    # "states" ,"actions" ,"rewards" ,"returns_to_go" ,"timesteps"

    def __call__(self, features):
        # add the returns to go

        batch_size = len(features)

        s, a, r, d, rtg, timesteps = [], [], [], [], [], []

        for i in range(batch_size):
            s.append(features[i]["states"])
            a.append(features[i]["actions"])
            r.append([features[i]["rewards"]])
            d.append(features[i]["dones"])
            rtg.append([TARGET_RETURN for _ in range(len(features[i]["rewards"]))])
            timesteps.append([i for i in range(len(features[i]["rewards"]))])

        s = torch.Tensor(s)
        a = torch.Tensor(a)
        r = torch.Tensor(r)
        d = torch.Tensor(d)
        rtg = torch.Tensor(rtg)
        timesteps = torch.Tensor(timesteps)

        # s = torch.from_numpy(np.concatenate(s, axis=0)).float()
        # a = torch.from_numpy(np.concatenate(a, axis=0)).float()
        # r = torch.from_numpy(np.concatenate(r, axis=0)).float()
        # d = torch.from_numpy(np.concatenate(d, axis=0))
        # rtg = torch.from_numpy(np.concatenate(rtg, axis=0)).float()
        # timesteps = torch.from_numpy(np.concatenate(timesteps, axis=0)).long()

        a = {
            "states": s,
            "actions": a,
            "rewards": r,
            "returns_to_go": rtg,
            "timesteps": timesteps,
        }

        return a

collator = MyDataCollator()

configuration = DecisionTransformerConfig(
	state_dim=state_size,
	act_dim=action_size,
)

model = DecisionTransformerModel(configuration)

training_args = TrainingArguments(
    output_dir="results_dt/",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    remove_unused_columns=False
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_ds["train"],
    # eval_dataset=ds["test"],
    data_collator=collator,
)

trainer.train()

  0%|          | 0/226 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (8x32 and 1x128)

  0%|          | 0/226 [00:00<?, ?it/s]

TypeError: list indices must be integers or slices, not str

In [ ]:

        # # add the timesteps
        # # features["timesteps"] = torch.tensor([
        # #     [i for i in range(len(features["rewards"][j]))]
        # #     for j in range(len(features["rewards"]))
        # #     ])
        # # features["returns_to_go"] = torch.tensor([TARGET_RETURN for _ in range(len(features["rewards"]))])

        # # # features["returns_to_go"] = torch.tensor([np.sum(features["rewards"][i:]) for i in range(len(features["rewards"]))])

        # # # add the mask
        # # # features["mask"] = torch.tensor([1 for _ in range(len(features["rewards"]))])

        # return features